<a href="https://colab.research.google.com/github/mprudd2/Project_Two_Team_Hodl/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/mprudd2/Project_Two_Team_Hodl.git

Cloning into 'Project_Two_Team_Hodl'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 20 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (20/20), done.


In [20]:
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import DateOffset


In [4]:
btc_df = pd.read_csv(
    Path("/content/Project_Two_Team_Hodl/Resources/Bitstamp_BTCUSD_d.csv"),
    index_col="date",
    infer_datetime_format=True,
    parse_dates=True
)



In [5]:
btc_df.head()

,symbol,open,high,low,close,Volume BTC,Volume USD
date,,,,,,,
2022-03-17,BTC/USD,41142.76,41187.60,40961.35,41049.85,8.817627,3.619623e+05
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07


In [6]:
btc_df["actual returns"] = btc_df["close"].pct_change()
btc_df.head()

,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2022-03-17,BTC/USD,41142.76,41187.60,40961.35,41049.85,8.817627,3.619623e+05,NaN
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08,0.002263
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07,-0.044724
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07,0.009236
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07,-0.047274


In [8]:
btc_df = btc_df.dropna()
display(btc_df.head())
display(btc_df.tail())

,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2022-03-16,BTC/USD,39302.80,41693.97,38865.67,41142.76,5987.103889,2.463260e+08,0.002263
2022-03-15,BTC/USD,39688.55,39896.13,38142.35,39302.70,1638.684030,6.440471e+07,-0.044724
2022-03-14,BTC/USD,37763.37,39913.42,37567.19,39665.71,1439.767682,5.710941e+07,0.009236
2022-03-13,BTC/USD,38798.68,39312.50,37595.62,37790.54,737.880132,2.788489e+07,-0.047274
2022-03-12,BTC/USD,38729.79,39458.58,38653.89,38822.02,962.855666,3.738000e+07,0.027295


,symbol,open,high,low,close,Volume BTC,Volume USD,actual returns
date,,,,,,,,
2014-12-02,BTC/USD,378.39,382.86,375.23,379.25,2593576.46,6832.53,0.006849
2014-12-01,BTC/USD,376.40,382.31,373.03,378.39,2520662.37,6660.56,-0.002268
2014-11-30,BTC/USD,376.57,381.99,373.32,373.34,1145566.61,3046.33,-0.013346
2014-11-29,BTC/USD,376.42,386.60,372.25,376.72,2746157.05,7245.19,0.009053
2014-11-28,BTC/USD,363.59,381.34,360.57,376.28,3220878.18,8617.15,-0.001168


In [9]:
short_window = 50

btc_df['sma_fast'] = btc_df['close'].rolling(window = short_window).mean()

In [10]:
long_window = 200

btc_df['sma_slow'] = btc_df['close'].rolling(window = long_window).mean()

In [11]:
btc_df = btc_df.dropna()

In [12]:
X = btc_df[['sma_fast', 'sma_slow']].shift().dropna().copy()


In [13]:
X.head()

,sma_fast,sma_slow
date,,
2021-08-28,49025.9800,48622.57310
2021-08-27,48773.9078,48661.53610
2021-08-26,48538.5124,48710.52505
2021-08-25,48242.4912,48746.47795
2021-08-24,48075.2946,48802.58400


In [14]:
X.tail()

,sma_fast,sma_slow
date,,
2014-12-02,301.8854,257.05375
2014-12-01,304.8986,257.72655
2014-11-30,308.2448,258.40140
2014-11-29,311.3740,259.02765
2014-11-28,314.6800,259.68235


In [15]:
btc_df['signal'] = 0.0

In [17]:
btc_df.loc[(btc_df['actual returns'] >= 0), 'signal'] = 1


btc_df.loc[(btc_df['actual returns'] < 0), 'signal'] = -1

In [18]:
y = btc_df['signal'].copy()

In [19]:
y.head()

date
2021-08-29    1.0
2021-08-28    1.0
2021-08-27    1.0
2021-08-26   -1.0
2021-08-25    1.0
Name: signal, dtype: float64

In [25]:

training_begin = X.index.min()


print(training_begin)




2014-11-28 00:00:00


In [31]:
training_end = X.index.min() + DateOffset(years=7)


print(training_end)

2021-11-28 00:00:00


In [34]:

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]




In [40]:
X_train.head()

,sma_fast,sma_slow
date,,
